# Actividad M3
#### Sebastián Márquez Álvarez A01632483
This projects looks to model thought a multi-agent system a traffic simulation.
To achieve this, 2 types of active agents have been implemented, a car, which is a reactive agent to
that complete disregards the current run and previous environment states and agent actions.
The second agent is a traffic light, which works by collaborating thought message passing. Having 2 possible
subtypes that change dynamically, being either a main light, or a follower, witch ask the main light for what state to
display or the main that goes green by perceiving a car, then goes on countdown for the state change.
## Imports

In [1]:
# Model design
import agentpy as ap

# Visualization
import matplotlib.pyplot as plt
import matplotlib.animation
import IPython
import numpy as np

from City import City, Direction, Orientation
from TrafficLightAgent import LightColor
from CityModel import CityModel

## Single Run Animation
### City definition

In [2]:
size = (20, 20)
city = City(*size)
city.add_street((4, 0), (4, size[0]), Direction.RIGHT)
city.add_street((8, 0), (8, size[0]), Direction.LEFT)
city.add_street((0, 4), (size[0], 4), Direction.UP)
city.add_street((0, 8), (size[0], 8), Direction.DOWN)
city.add_street((0, 12), (size[0], 12), Direction.UP)
city.add_street((0, 16), (size[0], 16), Direction.DOWN)

# Define parameters
parameters = {
    "size": size,
    "city": city,
    "cars": 49,
    "spawn_points": [(4, 0), (8, 19), (19, 4), (19, 8), (19, 12), (19, 16)],
    "steps": 200
}

### Create single-run animation with custom colors
(the horizontal traffic light, is the one displayed)

In [3]:
def map_agent_to_number(agents: ap.AgentIter):
    agent: ap.Agent
    road_exists = False
    for agent in agents:
        if agent.type == 'TrafficLightAgent' and agent.orientation == Orientation.HORIZONTAL:
            if agent.get_current_light() == LightColor.GREEN:
                return 2
            elif agent.get_current_light() == LightColor.YELLOW:
                return 3
            else:
                return 4
        if agent.type == 'CarAgent':
            return 1
        if agent.type == 'RoadAgent':
            road_exists = True
    if road_exists:  # There is road and no cars
        return 0
    return np.nan  # There are no agents in that position, use numpy.nan instead of None

def animation_plot(model, ax):
    attr_grid = model.grid.apply(map_agent_to_number)  # apply(map_agent_to_number)
    color_dict = {
        0: '#7777',
        1: '#d62c2c',
        2: '#00ff00',
        3: '#ffff00',
        4: '#ff0000',
        None: '#d5e5d5'
    }
    ap.gridplot(attr_grid, color_dict, True, ax)
    ax.set_title(f"Simulation of a city\n"
                 f"Time-step: {model.t}")

fig, ax = plt.subplots()
city_model = CityModel(parameters)
animation = ap.animate(city_model, fig, ax, animation_plot)
IPython.display.HTML(animation.to_jshtml(fps=15))